In [108]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from io import StringIO

In [109]:
def close_sign_in_if_open(driver):
    botton_signup = driver.find_element(By.XPATH, '//*[@id="PromoteSignUpPopUp"]/div[2]/i')
    if botton_signup.is_displayed():
        botton_signup.click()

In [110]:
service = Service(executable_path='../chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(f'https://www.investing.com/economic-calendar/')
time.sleep(1)

botton_cookies = driver.find_element(
    By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'
)
botton_cookies.click()
time.sleep(1)

close_sign_in_if_open(driver)
botton_yesterday = driver.find_element(
    By.XPATH, '//*[@id="timeFrame_yesterday"]'
)
botton_yesterday.click()
time.sleep(1)

In [111]:
close_sign_in_if_open(driver)
table = driver.find_element(
    By.XPATH, '//*[@class="eCalNew eCalMainNew ecoCalCont"]'
)
dfs = pd.read_html(StringIO(table.get_attribute('innerHTML')))
df_raw = dfs[0]
if df_raw.columns[0] is not str: # Fix for ocasional multi-index
    df_raw.columns = [c[0] for c in df_raw.columns]
df_raw

,Time,Cur.,Imp.,Event,Actual,Forecast,Previous,Unnamed: 7_level_0,Unnamed: 8_level_0
0,"Wednesday, October 18, 2023","Wednesday, October 18, 2023","Wednesday, October 18, 2023","Wednesday, October 18, 2023","Wednesday, October 18, 2023","Wednesday, October 18, 2023","Wednesday, October 18, 2023","Wednesday, October 18, 2023","Wednesday, October 18, 2023"
1,02:00,GBP,NaN,Core CPI (YoY) (Sep),6.1%,6.0%,6.2%,NaN,NaN
2,02:00,GBP,NaN,Core CPI (MoM) (Sep),0.5%,0.5%,0.1%,NaN,NaN
3,02:00,GBP,NaN,Core PPI Output (YoY) (Sep),0.7%,NaN,1.5%,NaN,NaN
4,02:00,GBP,NaN,Core PPI Output (MoM) (Sep),0.0%,NaN,-0.1%,NaN,NaN
...,...,...,...,...,...,...,...,...,...
84,20:30,AUD,NaN,Reserve Assets Total (Sep),93.2B,NaN,93.9B,NaN,NaN
85,20:30,AUD,NaN,Unemployment Rate (Sep),3.6%,3.7%,3.7%,NaN,NaN
86,21:00,KRW,NaN,Bank of Korea Monetary Policy Board’s Policy S...,NaN,NaN,NaN,NaN,NaN
87,21:00,KRW,NaN,Interest Rate Decision (Oct),3.50%,3.50%,3.50%,NaN,NaN


In [112]:
df_filtered = df_raw[~df_raw['Cur.'].isna()] # Exclude holidays
df_filtered.loc[:, 'Date'] = df_filtered['Imp.'].ffill() # Get dates for all rows
df_filtered = df_filtered[df_filtered['Imp.'].isna()] # Remove rows with dates

close_sign_in_if_open(driver)
# Read countries' names from flags (important for EUR)
flags = driver.find_elements(
    By.XPATH, '//*[@class="left flagCur noWrap"]/span'
)
countries = []
for flag in flags:
    countries.append(flag.get_attribute('title'))

df_filtered['Country'] = countries
df_filtered = df_filtered[~df_filtered['Actual'].isna()] # Get only indicators

df = df_filtered[['Time', 'Country', 'Cur.', 'Event',
                  'Actual', 'Forecast', 'Previous']]
df.index = pd.to_datetime(df_filtered['Date'])
df.index.name = None
df

,Time,Country,Cur.,Event,Actual,Forecast,Previous
2023-10-18,02:00,United Kingdom,GBP,Core CPI (YoY) (Sep),6.1%,6.0%,6.2%
2023-10-18,02:00,United Kingdom,GBP,Core CPI (MoM) (Sep),0.5%,0.5%,0.1%
2023-10-18,02:00,United Kingdom,GBP,Core PPI Output (YoY) (Sep),0.7%,NaN,1.5%
2023-10-18,02:00,United Kingdom,GBP,Core PPI Output (MoM) (Sep),0.0%,NaN,-0.1%
2023-10-18,02:00,United Kingdom,GBP,Core RPI (YoY) (Sep),7.6%,NaN,7.8%
...,...,...,...,...,...,...,...
2023-10-18,20:30,Australia,AUD,Participation Rate (Sep),66.7%,67.0%,67.0%
2023-10-18,20:30,Australia,AUD,Reserve Assets Total (Sep),93.2B,NaN,93.9B
2023-10-18,20:30,Australia,AUD,Unemployment Rate (Sep),3.6%,3.7%,3.7%
2023-10-18,21:00,South Korea,KRW,Interest Rate Decision (Oct),3.50%,3.50%,3.50%


In [113]:
driver.close()